In [3]:
import pandas as pd
import numpy as np

In [4]:
# load the weather table
weather_total = pd.read_csv('final_weather_table.txt',low_memory=False)
weather_total.head()

,Unnamed: 0,YYMM,State,WBAN,Latitude,Longitude,AvgTemp,TotalMonthlyPrecip,Avgwindspeed
0,0,199607,AK,25704,51.883,-176.650,51.6,0.22,NaN
1,1,199607,AL,13894,30.688,-88.246,81.7,4.95,5.6
2,2,199607,AL,13895,32.301,-86.394,80.8,8.72,4.8
3,3,199607,AL,3856,34.644,-86.786,78.2,4.7,5.6
4,4,199607,AR,13964,35.334,-94.365,79.7,7.67,5.3


In [5]:
# extract specific columns to form a new dataframe
# replace all "M" to nan and drop it
temp = weather_total.iloc[:,[1,2,3,-3]]
temp.replace('M',np.nan,inplace=True)
temp.dropna(inplace=True)

C:\Users\yongquan\Miniconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yongquan\Miniconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
# revise the type of columns
temp['YYMM'] = temp['YYMM'].astype(str)
temp['AvgTemp'] = temp['AvgTemp'].astype(float)

C:\Users\yongquan\Miniconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\yongquan\Miniconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
# For loop for all 51 states
statelist = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 
             'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 
             'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 
             'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
state_total = pd.DataFrame()

# Run for all 51 states
for item in statelist:
    name = item # remain the name of state as str
    item = temp[temp['State'].isin([item])] # extract dataframe of a specific state
    # run 1996-2016 of AZ and get the dataframe of TempSummer and TempWinter
    tempsw = pd.DataFrame()
    year_list = ['1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006',
                 '2007','2008','2009','2010','2011','2012','2013','2014','2015','2016']
    tempsummer_list = []
    tempwinter_list = []

    for i in year_list:
        year = item[item['YYMM'].str.contains(i)] # find all the data in a specific year
        YYMM_list = list(year['YYMM']) 
        month_index = []
        for i in YYMM_list:
            if not i in month_index:
                month_index.append(i)
    
        # run all the month in a specific year and get the dataframe（named avg_temp） 
        avg = []
        avg_temp = pd.DataFrame()
        for i in month_index:
            month_df = item[item['YYMM'].isin([i])]
            avg.append(sum(list(month_df['AvgTemp']))/len(month_df))
            avg_temp['YYMM'] = month_index
        avg_temp['AvgTemp'] = avg
    
        # find the tempsummer and tempwinter in a specific year for a state
        if len(avg_temp) == 12:
            tmax_index = avg_temp['AvgTemp'].idxmax()
            tempmax_three = 0
            for i in range(tmax_index-1,tmax_index+2):
                tempmax_three += avg_temp.iloc[i,-1]
            tempsummer = tempmax_three/3
    
            tmin_index = avg_temp['AvgTemp'].idxmin()
            tempmin_three = 0
            for i in range(tmin_index-1,tmin_index+2):
                if i>11:
                    i = i-12# index jump back to the first month
                tempmin_three += avg_temp.iloc[i,-1]
            tempwinter = tempmin_three/3
        # if there is not enough data for all 12 months, direcly use the Tmax as Tsummer,Tmin as Twinter 
        else:
            tempsummer = avg_temp['AvgTemp'].max()
            tempwinter = avg_temp['AvgTemp'].min()
    
        tempsummer_list.append(tempsummer)
        tempwinter_list.append(tempwinter)
 
    # create a list of state containing 21 items of AZ
    state = pd.Series(np.zeros(21))
    state.replace('0.0',name,inplace=True)

    tempsw['State'] = state
    tempsw['Year'] = year_list
    tempsw['TempSummer'] = tempsummer_list
    tempsw['TempWinter'] = tempwinter_list
    
    state_total = state_total.append(tempsw,ignore_index=True)

In [8]:
state_total

,State,Year,TempSummer,TempWinter
0,AK,1996,55.000000,31.320000
1,AK,1997,53.386970,27.537821
2,AK,1998,53.293705,17.716720
3,AK,1999,54.316565,8.305729
4,AK,2000,52.225000,20.123256
5,AK,2001,53.287839,15.282611
6,AK,2002,54.825000,13.759524
7,AK,2003,52.302326,12.611628
8,AK,2004,58.327273,15.888372
9,AK,2005,54.793909,16.910694


In [19]:
# export data to csv file
state_total.to_csv('state_total_temp.csv')

In [9]:
print (max(state_total.TempSummer))
print (min(state_total.TempSummer))
print (max(state_total.TempWinter))
print (min(state_total.TempWinter))

87.6416179337
50.4333333333
73.8114285714
0.0
